In [18]:
import re
import os

%pip install pandas
import pandas as pd

%pip install --upgrade pip

%pip install requests
import requests

%pip install bs4
from bs4 import BeautifulSoup

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [20]:
#setting working directory to eml_files
os.chdir('eml_files')

FileNotFoundError: [Errno 2] No such file or directory: 'eml_files'

In [ ]:
def urls_in_file(file_name):
    """ Takes an .eml file name as a string and returns a list of all unique urls linking to news sources """

    #creates file path by joining the current working directory with the file name
    file_path = os.path.join(os.getcwd(), file_name)

    #reads the file and removes newlines and equal signs (which are placed at the end of each line)
    with open(file_path, 'r') as file:
        textfile = file.read().replace('\n', '').replace('=', '')
    
    #any strings without white spaces starting with 3Dhttps:// and ending with &amp, non-greedy matching
    valid_urls = re.findall('3D(https://\S*?)?&amp', textfile)
    
    #creates a list of each unique url (removes duplicates)
    unique_urls = list(set(valid_urls))
    
    return unique_urls

#demonstrates the function with the first file in the folder
urls_in_file('1. El Salvador Mineria 1-94 eml/Google Alert - El Salvador minería_1.eml')

['https://www.elsalvador.com/noticias/nacional/mercados-mercado-cuscatlan-alcaldia-de-san-salvador-centro-/1209302/2025/',
 'https://www.elsalvador.com/opinion/editoriales/mineria-/1209390/2025/',
 'https://www.laprensagrafica.com/tendencias/Un-Pikachu-revolucionario-un-manifestante-disfrazado-de-Pokemon-se-unio-a-las-protestas-en-Turquia-20250329-0020.html',
 'https://www.laprensagrafica.com/opinion/Las-locuras-del-emperador-II-20250328-0081.html',
 'https://www.revistafactum.com/slm-grok/',
 'https://diario.elmundo.sv/el-mundo/un-pueblo-del-sur-de-honduras-podria-perder-su-historica-iglesia-por-la-explotacion-minera',
 'https://diario.elmundo.sv/nacionales/organizaciones-dicen-que-el-sistema-judicial-esta-a-prueba-tras-la-entrega-de-mas-de-59000-firmas-contra-de-la-mineria']

In [ ]:
def urls_in_folder(folder_name):
    """ Takes a folder of .eml files as a string and returns a list of all unique urls contained in each of the 
    files in the folder 
"""
    
    #creates a list of all file names in folder_name
    filenames = os.listdir(folder_name)
    filenames = [os.path.join(folder_name, file) for file in filenames]
    
    #creates a list to store the unique urls found in each file using urls_in_file function
    all_urls = []
    for file in filenames:
        urls = urls_in_file(file)
        all_urls.extend(urls)
    
    return all_urls

#demonstrates the function with folder '1. El Salvador Mineria 1-94 eml'
urls_in_folder('1. El Salvador Mineria 1-94 eml')[:5]

In [ ]:
def extract_website_text(url):
    """Takes in a url as a string and returns the text content of the webpage, including headers, footers, and paragraphs."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        extracted_text = soup.get_text(separator='\n', strip=True)
        return extracted_text
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None
    
def clean_extracted_text(text, section='body', cutoff=20):
    """Cleans the text extracted using extract_website_text and removes short paragraphs (headers, footers, etc...) based on word count cutoff"""
    if text is None:
        return None
    elif section == 'body':
        split = text.split('\n') #split the text into a list divided every time there is a new line \n
        body = [paragraph for paragraph in split if len(paragraph.split()) > cutoff] #remove paragraphs with less than CUTOFF words.
        body_text = ' '.join(body) #combine remaining text into a single string
        print(body_text)
    elif section == 'header':
        # Assuming the header is the first part of the text before the first paragraph
        header = text.split('\n')[0] if '\n' in text else text.split('-')[0]
        return header.strip()


example_text = extract_website_text('https://www.contrapunto.com.sv/muro-colapso-tras-sismo-en-san-salvador-y-dejo-a-una-persona-herida/')
clean_extracted_text(example_text, section='body')

In [21]:
def extract_info_from_emls_in_folder(folder_name):
    """Extracts text from all .eml files in a folder and returns a table filled with urls, article titles, and body text."""
    
    urls = urls_in_folder(folder_name)
    
    raw_url_text = [extract_website_text(url) for url in urls]
    cleaned_text = [clean_extracted_text(text, section='body') for text in raw_url_text]
    extracted_info = pd.DataFrame({
        'url': urls_in_folder,
        'title': [clean_extracted_text(text, section='header') for text in raw_url_text],
        'body': cleaned_text
    })
    
    return extracted_info

#mineria_1 = extract_info_from_emls_in_folder('1. El Salvador Mineria 1-94 eml')
#mineria_1

In [ ]:
def emls_in_file_to_table(file_name):
    """Extracts text from the file_name in a folder and returns a table filled with urls, article titles, and body text."""
    
    urls = urls_in_folder(file_name)
    
    raw_url_text = [extract_website_text(url) for url in urls]
    cleaned_text = [clean_extracted_text(text, section='body') for text in raw_url_text]
    extracted_info = pd.DataFrame({
        'url': urls_in_folder,
        'title': [clean_extracted_text(text, section='header') for text in raw_url_text],
        'body': cleaned_text
    })
    
    return extracted_info

mineria_2 = emls_in_file_to_table('Google Alert - El Salvador minería_1.eml')
mineria_2

NotADirectoryError: [Errno 20] Not a directory: '1. El Salvador Mineria 1-94 eml/Google Alert - El Salvador minería_1.eml'